In [1]:
# import necessary packages

import torch
import torch.nn as nn
import torch.optim as optim

import os
import numpy as np
import random
import pandas as pd

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt


# set random seed

seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# make sure cuda works
print(device)

cuda


In [2]:
# Consider a larger problem
from scipy.stats import bernoulli, norm, multivariate_normal

problem_size = 500
problem_dim = 20
noise_amp = 0.01

nomial_sparsity = 3

X = np.random.normal(size=(problem_size, problem_dim))

# generate a sparse normal
sparse_vector = np.zeros(20)
random_numbers = np.random.uniform(low=1.0, high=10.0, size=nomial_sparsity)
indices = np.random.choice(range(20), size=nomial_sparsity, replace=False)
sparse_vector[indices] = random_numbers
beta_true = sparse_vector


#beta_0_true = np.ones(20)*5.0
#sigma_0_true = np.eye(20)*2

#beta_true = multivariate_normal.rvs(mean = beta_0_true, cov = sigma_0_true)
print("True betas are: ", beta_true)

noise = np.random.normal(loc = 0.0, scale = noise_amp, size = problem_size)

y = X @ beta_true #+ noise
#print(y.shape, X.shape, beta_true.shape, (X @ beta_true).shape, noise.shape)

True betas are:  [0.         0.         0.         0.         0.         0.
 0.         0.         1.92717275 0.         0.         0.
 0.         3.89129656 0.         0.         1.64626791 0.
 0.         0.        ]


In [3]:
def gibbs_spike_slab(y, X, num_iters, theta, tau_inv, sigma_squared):
    
    n, p = X.shape
    
    # Allocate space for trace
    trace = np.empty((num_iters, p))
    
    # Compute constant values for the entire gibbs sampling
    # Constant values for gibbs sampling
    sigma_hat = np.zeros((p, p))  # initialize sigma with zeros
    mu_hat = np.zeros(p)  # initialize the result with zeros
    
    for t in range(n):
        row = X[t, :]
        sigma_hat += np.outer(row, row.T)  # add the outer product of the row to sigma
        mu_hat += row * y[t] # multiply the t-th row of X by the t-th entry of y and add to the result

    # Initialize Z and beta
    Z = np.array(bernoulli.rvs(theta, size=20))
    beta = np.array(norm.rvs(loc = 0, scale = tau_inv, size = 20))

    tau = 1/tau_inv

    for i in range(num_iters):
    
        for i_dim in range(p):
            
            curr_prob = Z[i_dim]
            hadamard_prod_Z = np.multiply(Z, beta)
            
            # log-sum-exp introduced by Wessel Bruinsma
            term_1_posdraw = np.log(theta)
            term_1_negdraw = np.log(1-theta)
            term_2 = (1/sigma_squared)*np.dot(hadamard_prod_Z, mu_hat)
            term_3 = (0.5/sigma_squared)*np.dot(hadamard_prod_Z, np.dot(sigma_hat,hadamard_prod_Z))
            
            nomial_prob_pos =  term_1_posdraw + term_2 - term_3
            nomial_prob_neg =  term_1_negdraw + term_2 - term_3
            
            nomial_prob_max = max(nomial_prob_pos, nomial_prob_neg)
            v_1 = nomial_prob_pos - nomial_prob_max
            v_0 = nomial_prob_neg - nomial_prob_max
            
            new_prob = np.exp(v_1) / (np.exp(v_0) + np.exp(v_1))
            
            Z[i_dim] = bernoulli.rvs(new_prob)
            
        # now that we have updated Z, compute constant value in this draw
        sigma_tilde = np.zeros((problem_dim, problem_dim))
        mu_tilde = np.zeros(problem_dim)

        for t in range(n):
            row = X[t, :]
            hadamard_prod_z = np.multiply(Z, row)
            sigma_tilde += np.outer(hadamard_prod_z, hadamard_prod_z.T)
            mu_tilde += hadamard_prod_z * y[t]
        
       
        
        inv_chunk = np.linalg.inv(tau*sigma_squared*np.eye(p) + sigma_tilde)
        M_beta = np.dot(inv_chunk, mu_tilde)
        V_beta = inv_chunk*sigma_squared
        #print(M_beta)
        #print(V_beta)
        beta = multivariate_normal.rvs(M_beta, V_beta)
        uncertainty = V_beta
        
        trace[i] = beta
    
    return trace

In [4]:
# Define initialization and prior for Gibbs sampling
num_samples = 5000

# Define spike and slab parameters
theta = 0.95  # Bernoulli probability for Z
sigma_squared = 0.01

# greater tau_inv smaller trash from sampling beta
tau_inv = 0.01

result = gibbs_spike_slab(y, X, num_samples, theta, tau_inv, sigma_squared)

In [5]:
print(result)

[[-3.63277254e-03 -2.83451392e-03  1.28310379e-03 ...  9.52086110e-04
  -6.80326544e-03  1.43937985e-03]
 [ 2.99275178e-03 -7.02986331e-04  4.79943786e-04 ...  9.65355286e-02
   2.44316988e-03  4.24700918e-03]
 [-5.85112532e-03 -5.41472249e-03  5.32446186e-03 ...  3.69304733e-03
   1.07340778e-02  1.55040495e-02]
 ...
 [-7.46412396e-03 -8.02492873e-03  2.54975435e-03 ...  8.14020560e-03
   2.13256573e-03 -1.28468167e-03]
 [ 1.00205218e-02  1.59134701e-01 -2.33322248e-02 ...  3.10603469e-02
   7.39285848e-02  9.09957595e-02]
 [ 5.65588959e-03 -1.76728319e-04 -8.87841788e-05 ... -5.15725378e-03
  -2.19855077e-03  5.15715769e-04]]


In [6]:
def threshold_vector(X, Threshold):
   
    X[np.abs(X) < Threshold] = 0
    return X

In [7]:
print(threshold_vector(result[-1], 1e-2))

[0.         0.         0.         0.         0.         0.
 0.         0.         1.92574084 0.         0.         0.
 0.         3.88016631 0.         0.         1.6415383  0.
 0.         0.        ]
